# Selenium
**Selenium** es una _Web Browser Automation Tool_, es decir, permite abrir un buscador y realizar tareas que "imiten" el comportamiento humano (clicks, scrolling, *búsquedas*, etc).  
Link a la documentación de Selinium: [https://selenium-python.readthedocs.io/](https://selenium-python.readthedocs.io/)  
Link a la dicumentación y descarga de chromedriver: [https://chromedriver.chromium.org/home](https://chromedriver.chromium.org/home)

In [8]:
# !pip install selenium

In [9]:
# !apt-get install -y chromium-browser

In [10]:
""" !apt-get update
# !apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') """

" !apt-get update\n# !apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\nimport sys\nsys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') "

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import pandas as pd
import time

In [12]:
from selenium import webdriver

def configure_driver():
    # Add additional Options to the webdriver
    options = webdriver.ChromeOptions()
    # add the argument and make the browser Headless.
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox') # only colab
    # options.add_argument('--disable-dev-shm-usage') # only colab
    # Handle the service
    # service = Service(executable_path=r'/usr/bin/chromedriver')
    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    driver = webdriver.Chrome(options=options)

    #driver = webdriver.Firefox()
    return driver

In [13]:
def getText(driver, total_clicks):
    # https://www.pluralsight.com/search?q=web%20scraping&categories=course
    # Step 1: Go to pluralsight.com, category section with selected search keyword
    # driver.get("https://www.pluralsight.com/search?q={search_keyword}&categories=course")
    # driver.get("https://booknet.com/es/reader/una-pequena-promesa-b425858")
    driver.get("https://booknet.com/es/reader/un-amor-para-april-b425666")

    timeout = 3
    count = 0
    loop_obj = tqdm(range(total_clicks))

    data = []
    for k in loop_obj:
        # loop_obj.set_description(f"Epoch: {k + 1}")
        loop_obj.set_postfix_str(f"Count error: {count}")

        # wait for the element to load
        try:
            WebDriverWait(driver, timeout*10).until(lambda s: s.find_element(By.CLASS_NAME, "reader-text").is_displayed())
        except TimeoutException:
            print("Timeout :(")
            return None
        
        # Step 2: Create a parse tree of page sources after searching
        soup = BeautifulSoup(driver.page_source, "lxml")

        # Step 3: Iterate over the search result and fetch the course
        content = soup.select("div.jsReaderText")

        try:
            title = soup.find("h2").text
        except:
            pass

        data.append(title)
        for course_page in content:
            for course in course_page.select("p"):
                data.append(course.text)
        
        try:
            element = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, "//a[@id='link-right']")))
            element.click()
            count = 0
        except TimeoutException as r:
            count += 1
            if count > 5:
                break
        time.sleep(timeout)
    return data

total_clicks = 500
# create the driver object.
driver = configure_driver()
data = getText(driver, total_clicks)

  7%|▋         | 34/500 [02:46<38:05,  4.90s/it, Count error: 5]


In [14]:
# close the driver.
driver.close()

In [51]:
file = open("book1.txt", "w")
for line in tqdm(data):
    file.write("{} \n".format(line))
file.close()

100%|██████████| 1271/1271 [00:00<00:00, 1214895.26it/s]


---

In [52]:
import nltk
nltk.download("punkt")

book = open("book1.txt", "r")

[nltk_data] Downloading package punkt to /home/win7/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
text_data = []
for line in book:
    text_data.append(line)

Remove punctuation

In [54]:
# Load libraries
import string
import numpy as np

In [55]:
# Create function using string.punctuation to remove all punctuation
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))

# Apply function
data_x = [remove_punctuation(sentence) for sentence in text_data]
data_x

['Capítulo 1 Apirl \n',
 'Aidan y Cian se toman las manos para caminar juntos Yo sonrío como la tía tonta y babosa que soy por sus sobrinos de un año que están aprendiendo a caminar \n',
 'A pesar de ser pequeños los dos tienen personalidades muy marcadas Aidan es más simpático y sociable que su gemelo Mientras que Cian desconfía bastante de las personas que intentan acercarse \n',
 'Ni siquiera cuando su hermano abre los brazos para saludar a cualquier persona que le brinde atención se acerca \n',
 'No hay dudas que Aidan es como Arizona y Cian tiene mucho de Troy aunque en el físico son copias de su padre o eso dice su abuelo paterno \n',
 '—Eso es vengan con la tía April \n',
 'Los dos llegan hasta mí los felicito y aplauden emocionados por mis felicitaciones Aidan se queda aferrado a mi mano mientras que Cian se suelta y por su sonrisa no necesito voltear para saber que mi hermana o su esposo se están acercando \n',
 'Me incorporo con Aidan en brazos y sonrío a mi hermana y cuñado 

---

In [19]:
book_sentences = nltk.sent_tokenize(book)
len(book_sentences)
book_sentences[1]

'Yo sonrío como la tía tonta y babosa que soy por sus sobrinos de un año que están aprendiendo a caminar.'

In [20]:
porter = nltk.stem.PorterStemmer()

In [ ]:
all_tokens = []
stem_tokens = [] # list tokens

for sent in book_sentences:
    for word in nltk.word_tokenize(sent):
        all_tokens.append(word)
        stem_tokens.append(porter.stem(word))

---